In [17]:
pip install ortools

In [18]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

In [19]:
matrix_1 = [
  [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
  [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
  [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
  [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
  [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
  [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
  [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
  [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
  [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
  [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
  [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
  [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
  [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
]
dist_function_1 = lambda a, b : matrix_1[a][b]
locations_1 = list(range(len(matrix_1)))
capacities_1 = [100, 100, 100, 100]
demands_1 = [0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

matrix_2 = [[0, 1, 1], [1, 0, 1], [1, 1, 0]]
dist_function_2 = lambda a, b : matrix_2[a][b]
locations_2 = list(range(len(matrix_2)))
capacities_2 = [(10, 0), (0, 10)]
demands_2 = [(0, 0), (1, 0), (0, 1)]

To change the start and end locations, change 
```python
manager = pywrapcp.RoutingIndexManager(len(time_matrix), no_vehicles, depot)
```
to
```python
manager = pywrapcp.RoutingIndexManager(len(time_matrix), no_vehicles, start, end)
```

</br>

If takes too long can set solution strategy to different one: https://developers.google.com/optimization/routing/routing_options#first_sol_options

</br>

Solver statuses: https://developers.google.com/optimization/routing/routing_options#search-status


# TSP

In [20]:
def tsp(dist_function, locations):  # function takes loc 1 and loc 2 and returns dist between

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), 1, 0)

  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  visited = []
  index = routing.Start(0)
  while not routing.IsEnd(index):
    visited.append(manager.IndexToNode(index))
    index = solution.Value(routing.NextVar(index))
  visited.append(manager.IndexToNode(index))
  
  return visited, solution.ObjectiveValue()

In [21]:
tsp(dist_function_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

# VRP

In [22]:
def vrp(dist_matrix, locations, no_vehicles, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, int(1e9), True, "Distance")
  distance_dimension = routing.GetDimensionOrDie("Distance")
  distance_dimension.SetGlobalSpanCostCoefficient(100)
  
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)
    
  return visited, max_route_distance

In [23]:
vrp(matrix_1, locations_1, 2)

([[0, 10, 5, 11, 1, 4, 2, 7, 0], [0, 3, 6, 8, 12, 9, 0]], 5728)

# Capacity constraints

In [24]:
def cvrp(dist_matrix, locations, no_vehicles, demands, capacities, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations) == len(demands)
  assert len(capacities) == no_vehicles

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)  # add more of these for different capacities
  routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, capacities, True, 'Capacity')
  
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  search_parameters.time_limit.FromSeconds(1)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  loads = [0 for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] += demands[manager.IndexToNode(index)]
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads

In [25]:
cvrp(matrix_1, locations_1, len(capacities_1), demands_1, capacities_1)

([[0, 0], [0, 0], [0, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], [0, 2, 7, 0]],
 6892,
 [0, 0, 100, 20])

# Pickups and Deliveries

In [26]:
matrix = [[0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
          [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
          [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
          [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
          [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
          [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
          [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
          [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
          [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
          [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
          [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
          [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
          [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
          [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
          [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
          [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
          [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0]]
routes = [[1, 6], [2, 10], [4, 3], [5, 9], [7, 8], [15, 11], [13, 12], [16, 14]]

In [27]:
def pickups_deliveries(dist_matrix, locations, no_vehicles, routes, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, int(1e9), True, "Distance")
  distance_dimension = routing.GetDimensionOrDie("Distance")
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  for request in routes:
    pickup_index = manager.NodeToIndex(request[0])
    delivery_index = manager.NodeToIndex(request[1])
    routing.AddPickupAndDelivery(pickup_index, delivery_index)
    routing.solver().Add(routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
    routing.solver().Add(distance_dimension.CumulVar(pickup_index) <= distance_dimension.CumulVar(delivery_index))

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  routes = [[] for __ in range(no_vehicles)]
  distances = []
  for vehicle_id in range(no_vehicles):
    index = routing.Start(vehicle_id)
    route_distance = 0
    while not routing.IsEnd(index):
      routes[vehicle_id].append(manager.IndexToNode(index))
      previous_index = index
      index = solution.Value(routing.NextVar(index))
      route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
    routes[vehicle_id].append(manager.IndexToNode(index))
    distances.append(route_distance)
  
  return routes, distances

In [28]:
pickups_deliveries(matrix, list(range(len(matrix))), 4, routes)

([[0, 5, 2, 10, 16, 14, 9, 0],
  [0, 7, 1, 6, 8, 0],
  [0, 13, 15, 11, 12, 0],
  [0, 4, 3, 0]],
 [2192, 1780, 1552, 1392])

# Time windows

In [29]:
matrix = [[0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
          [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
          [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
          [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
          [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
          [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
          [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
          [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
          [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
          [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
          [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
          [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
          [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
          [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
          [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
          [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
          [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0]]

time_windows = [(0, 5), (7, 12), (10, 15), (16, 18), (10, 13), (0, 5), (5, 10), (0, 4), (5, 10),
                (0, 3), (10, 16), (10, 15), (0, 5), (5, 10), (7, 8), (10, 15), (11, 15)]

In [30]:
def vrptw(time_matrix, locations, no_vehicles, time_windows, depot=0, time_limit=-1):

  assert len(time_matrix) == len(time_matrix[0]) == len(locations) == len(time_windows)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return time_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(time_matrix), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(time_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, int(1e9), int(1e9), False, "Time")
  time_dimension = routing.GetDimensionOrDie("Time")

  
  for location_idx, time_window in enumerate(time_windows):
    if location_idx == 0:
      continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      time_dimension.CumulVar(index).SetRange(time_windows[0][0], time_windows[0][1])

  for i in range(no_vehicles):
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  routes = [[] for i in range(no_vehicles)]
  times = [0 for i in range(no_vehicles)]

  time_dimension = routing.GetDimensionOrDie('Time')
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      while not routing.IsEnd(index):
          time_var = time_dimension.CumulVar(index)
          routes[vehicle_id].append(manager.IndexToNode(index))
          index = solution.Value(routing.NextVar(index))
      time_var = time_dimension.CumulVar(index)
      routes[vehicle_id].append(manager.IndexToNode(index))
      times.append(solution.Min(time_var))

  return routes, times

In [31]:
vrptw(matrix, list(range(len(matrix))), 4, time_windows)

([[0, 9, 14, 16, 0],
  [0, 7, 1, 4, 3, 0],
  [0, 12, 13, 15, 11, 0],
  [0, 5, 8, 6, 2, 10, 0]],
 [0, 0, 0, 0, 18, 24, 20, 20])

# Load and Unload times

In [32]:
matrix = [[0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0]]
        
time_windows = [(0, 5), (7, 12), (10, 15), (5, 14), (5, 13), (0, 5), (5, 10), (0, 10), (5, 10),
                (0, 5), (10, 16), (10, 15), (0, 5), (5, 10), (7, 12), (10, 15), (5, 15)]

In [33]:
def load_unload(time_matrix, locations, no_vehicles, time_windows, load_time, unload_time, depot_capacity, depot=0, time_limit=-1):

  assert len(time_matrix) == len(time_matrix[0]) == len(locations) == len(time_windows)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return time_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(time_matrix), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(time_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, int(1e9), int(1e9), False, "Time")
  time_dimension = routing.GetDimensionOrDie("Time")
  
  for location_idx, time_window in enumerate(time_windows):
      if location_idx == 0:
          continue
      index = manager.NodeToIndex(location_idx)
      time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      time_dimension.CumulVar(index).SetRange(time_windows[0][0], time_windows[0][1])

  solver = routing.solver()
  intervals = []
  for i in range(no_vehicles):
    intervals.append(solver.FixedDurationIntervalVar(time_dimension.CumulVar(routing.Start(i)), load_time, 'depot_interval'))
    intervals.append(solver.FixedDurationIntervalVar(time_dimension.CumulVar(routing.End(i)), unload_time, 'depot_interval'))

  depot_usage = [1 for i in range(len(intervals))]
  solver.Add(solver.Cumulative(intervals, depot_usage, depot_capacity, 'depot'))

  for i in range(no_vehicles):
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  routes = [[] for i in range(no_vehicles)]
  times = []

  time_dimension = routing.GetDimensionOrDie('Time')
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      while not routing.IsEnd(index):
          time_var = time_dimension.CumulVar(index)
          routes[vehicle_id].append(manager.IndexToNode(index))
          index = solution.Value(routing.NextVar(index))
      time_var = time_dimension.CumulVar(index)
      routes[vehicle_id].append(manager.IndexToNode(index))
      times.append(solution.Min(time_var))

  return routes, times

In [34]:
load_unload(matrix, list(range(len(matrix))), 4, time_windows, 5, 5, 2)

([[0, 8, 14, 16, 0],
  [0, 12, 13, 15, 11, 0],
  [0, 7, 1, 4, 3, 0],
  [0, 9, 5, 6, 2, 10, 0]],
 [20, 20, 25, 25])

# Penalites

In [35]:
matrix = [[0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
        [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
        [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
        [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
        [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
        [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
        [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
        [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
        [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
        [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
        [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
        [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
        [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
        [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
        [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
        [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
        [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],]
demands = [0, 1, 1, 3, 6, 3, 6, 8, 8, 1, 2, 1, 2, 6, 6, 8, 8]
capacities = [15, 15, 15, 15]

In [36]:
def cvrp_penalty(dist_matrix, locations, no_vehicles, demands, capacities, drop_penalty, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations) == len(demands)
  assert len(capacities) == no_vehicles

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return demands[from_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
  routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, capacities, True, 'Capacity')

  for node in range(1, len(dist_matrix)):
    routing.AddDisjunction([manager.NodeToIndex(node)], drop_penalty)
  
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
  search_parameters.time_limit.FromSeconds(1)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  loads = [0 for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] += demands[manager.IndexToNode(index)]
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads

In [37]:
cvrp_penalty(matrix, list(range(len(matrix))), len(capacities), demands, capacities, 1000)

([[0, 8, 14, 9, 0],
  [0, 1, 3, 4, 11, 12, 0],
  [0, 7, 13, 0],
  [0, 5, 6, 2, 10, 0]],
 1872,
 [15, 13, 14, 12])

# tsp/vrp/cvrp combined general function (Not working :[  )

In [38]:
def routing(dist_function, locations, no_vehicles=1, capacities=[], demands=[], time_windows=[], max_travel_dist=int(1e9), time_limit=-1):

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  if capacities == []:  # if no capacity entered assume infinite capacity
    capacities = [(int(1e9),) for __ in range(no_vehicles)]
  if demands == []:  # if no demand entered assume 1 demand
    demands = [(1,) for __ in range(len(locations))]

  if type(demands[0]) != tuple:
    capacities = [(i,) for i in capacities]
    demands = [(i,) for i in demands]

  if len(time_windows) == 0:
    time_windows = [(0, max_travel_dist) for __ in range(len(locations))]

  assert len(capacities) == no_vehicles
  assert len(demands) == len(locations) == len(time_windows)

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  def demand_callback(from_index, idx):  # callback for material: idx
    from_node = manager.IndexToNode(from_index)
    return demands[from_node][idx]

  def add(a, b):
    return [a[i]+b[i] for i in range(len(a))]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), no_vehicles, 0)
  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, max_travel_dist, True, 'Distance')  # For multiple vehicles
  distance_dimension = routing.GetDimensionOrDie('Distance')
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  for i in range(len(capacities[0])):  # add dimension for each capacity
    demand_callback_index = routing.RegisterUnaryTransitCallback(lambda x : demand_callback(x, i))
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [j[i] for j in capacities], True, f'Capacity_{i}')

  routing.AddDimension(transit_callback_index, int(1e9), int(1e9), False, "Time")  # For time constraints
  time_dimension = routing.GetDimensionOrDie("Time")
  for location_idx, time_window in enumerate(time_windows):
    if location_idx == 0:
      continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
  for vehicle_id in range(no_vehicles):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(time_windows[0][0], time_windows[0][1])
  for i in range(no_vehicles):
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))
    
  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  loads = [[0 for __ in range(len(capacities[0]))] for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] = add(loads[vehicle_id], demands[manager.IndexToNode(index)])
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads  # route (vehicles, location idx), cost

In [39]:
def tsp(dist_function, locations):
  x = routing(dist_function, locations)
  return x[0][0], x[1]

In [40]:
def vrp(dist_function, locations, no_vehicles):
  x = routing(dist_function, locations, no_vehicles=no_vehicles)
  return x[:2]

In [41]:
def cvrp(dist_function, locations, no_vehicles, capacities, demands):
  x = routing(dist_function, locations, no_vehicles=no_vehicles, capacities=capacities, demands=demands)
  if len(x[2][0]) == 1:
    return x[0], x[1], [i[0] for i in x[2]]
  return x[0], x[1], x[2]

In [42]:
tsp(dist_function_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

In [43]:
vrp(dist_function_1, locations_1, 2)  # not working because have multiple things to maximise?

SystemError: ignored

In [44]:
cvrp(dist_function_1, locations_1, len(capacities_1), capacities_1, demands_1)

SystemError: ignored

In [45]:
cvrp(dist_function_2, locations_2, len(capacities_2), capacities_2, demands_2)

ValueError: ignored

# Modular thing (Not working :[  )

Attempt to use a vase vrp and then allow dimensions to be added to upgrade to different problem through function input

In [46]:
def vrp_base(dist_matrix, locations, no_vehicles, update_routing=lambda x : None, depot=0, time_limit=-1):

  assert len(dist_matrix) == len(dist_matrix[0]) == len(locations)

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  manager = pywrapcp.RoutingIndexManager(len(locations), no_vehicles, depot)
  routing = pywrapcp.RoutingModel(manager)

  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, int(1e9), True, "Distance")
  distance_dimension = routing.GetDimensionOrDie("Distance")
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  update_routing(routing)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  return solution, routing, manager

In [47]:
def tsp(dist_matrix, locations, depot=0, time_limit=-1):
  solution, routing, manager = vrp_base(dist_matrix, locations, 1, depot=depot, time_limit=time_limit)

  visited = []
  index = routing.Start(0)
  route_distance = 0
  while not routing.IsEnd(index):
    visited.append(manager.IndexToNode(index))
    previous_index = index
    index = solution.Value(routing.NextVar(index))
    route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
  visited.append(manager.IndexToNode(index))
  
  return visited, route_distance

In [48]:
def vrp(dist_matrix, locations, no_vehicles, depot=0, time_limit=-1):

  solution, routing, manager = vrp_base(dist_matrix, locations, no_vehicles, depot=depot, time_limit=time_limit)

  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)
    
  return visited, max_route_distance

In [49]:
def cvrp(dist_matrix, locations, no_vehicles, demands, capacities, depot=0, time_limit=-1):

  if type(demands[0]) != tuple:
    demands = [(i,) for i in demands]
    capacities = [(i,) for i in capacities]

  assert len(demands) == len(locations)
  assert len(capacities) == no_vehicles

  def demand_callback(from_index, cargo_type):
    from_node = manager.IndexToNode(from_index)
    return demands[cargo_type][from_node]

  def update_routing(routing):
    for i in range(len(demands[0])):
      callback = lambda x : demand_callback(x, i)
      demand_callback_index = routing.RegisterUnaryTransitCallback(callback)
      routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [j[i] for j in capacities], True, f'Capacity_{i}')

  solution, routing, manager = vrp_base(dist_matrix, locations, no_vehicles, update_routing=update_routing, depot=depot, time_limit=depot)

  loads = [[0 for __ in range(len(capacities[0]))] for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] = add(loads[vehicle_id], demands[manager.IndexToNode(index)])
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads

In [50]:
def vrptw():
  pass

In [51]:
tsp(matrix_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

In [52]:
vrp(matrix_1, locations_1, 2)

([[0, 10, 5, 11, 1, 4, 2, 7, 0], [0, 3, 6, 8, 12, 9, 0]], 5728)

In [53]:
cvrp(matrix_1, locations_1, len(capacities_1), demands_1, capacities_1)

SystemError: ignored

In [54]:
cvrp(dist_function_2, locations_2, len(capacities_2), capacities_2, demands_2)  # WHY DOESNT THIS WORK ??!!!

AssertionError: ignored